### Cargamos el Dataset de MINST

In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Transformación de imagen a tensor
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Datasets
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset  = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

X_train, y_train = train_dataset.data.numpy(), train_dataset.targets.numpy()
X_test, y_test = test_dataset.data.numpy(), test_dataset.targets.numpy()

X_train, X_test = X_train.reshape(len(X_train), -1) / 255, X_test.reshape(len(X_test), -1) / 255

/home/jesus/miniconda3/envs/entorno_universidad/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/jesus/miniconda3/envs/entorno_universidad/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


### Cargamos los modelos que hemos entrenado y guardado en los archivos python.

In [2]:
import torch
import joblib
from NN import NN

red = NN(28*28)
red.load_state_dict(torch.load('modelo.pth', map_location= torch.device('cpu')))
red.eval()

svc = joblib.load('SVC.pkl')

### Comprobamos el accuracy de estos mdoelos.

In [3]:
from sklearn.metrics import accuracy_score

print(f'Accuracy red: {red.accuracy(test_dataset, device= "cpu")}')
print(f'Accuracy SVC: {accuracy_score(y_test, svc.predict(X_test))}')

Accuracy red: 0.9841
Accuracy SVC: 0.9833


### Generamos unas cuantas imagenes adversarias y tratamos de predecirlas con la red y con la SVC.

In [ ]:
from FGSM import get_adversarial_images
import matplotlib.pyplot as plt

epsilons = [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]
accuracies_red = []
accuracies_svc = []

test_dataset_loader = DataLoader(test_dataset, batch_size= 256)

for epsilon in epsilons:

        modified_images, labels = get_adversarial_images(red, test_dataset_loader, epsilon= epsilon)
        # Primero lo hacemos para la red neuronal
        with torch.no_grad():
            outputs = red(modified_images)
            y_pred = torch.argmax(outputs, dim= 1)
            total = labels.size(0)
            correct = (y_pred == labels).sum().item()
        
        # Calculamos el accuracy.
        accuracies_red.append(correct / total)
        
		# Ahora para la svc
        y_pred = svc.predict(modified_images.detach().numpy().reshape(len(modified_images), 28*28))
        accuracies_svc.append(y_pred == labels.numpy() / total)

plt.title('Relación accuracy valor de epsilon Red Neuronal')
plt.plot(epsilons, accuracies_red, color= 'red', label= 'Accuracy red')
plt.plot(epsilons, accuracies_svc, color= 'blue', label= 'Accuracy svc')
plt.xlabel('Epsilon')
plt.ylabel('Accuracy')